- Related works:
    - Dialogue-Act Prediction of Future Responses based on Conversation History (http://aclanthology.lst.uni-saarland.de/P19-2027.pdf)
    - DeliData: A dataset for deliberation in multi-party problem solving (https://arxiv.org/pdf/2108.05271.pdf)
    - Identifying Agreement/Disagreement in Conversational Speech: A Cross-lingual Study (https://www.sri.com/wp-content/uploads/2021/12/identifying_agreement-disagreement_in_conversational_speech.pdf)
    - The Role of Conversational Structure in Agreement and Disagreement Detection in Online Discussions (https://aclanthology.org/W15-4625.pdf)
    - Modeling Long-Range Context for Concurrent Dialogue Acts Recognition https://arxiv.org/pdf/1909.00521.pdf
    - User Satisfaction Estimation with Sequential Dialogue Act Modeling in Goal-oriented Conversational Systems https://arxiv.org/pdf/2202.02912.pdf
    - 

- Experiments:
    - Predict understanding vs non-understanding signals in dialogue -> That indicates quality of explanation (baseline)
    - LongFormer that consumes all the text up to window_size and predict the quality
    - Modeling quality by the degree of non-understanding explanation_moves in the dialogue: Average probability of all uternaces of being a non-understanding explanation move.
    - RQ: Can we predict ahead whether the dialogue will lead to non-understanding?

In [8]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
import os
import wandb

sys.path.insert(0, '../src-py')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#wandb.init(project="test-project", entity="milad-it")

In [10]:
import json
import pandas as pd
import numpy as np
import glob 
from tabulate import tabulate
import math
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


pd.set_option('display.max_colwidth', None)

In [11]:
eli5_annotation_df  = pd.read_pickle('../../data/eli5_ds/annotation-results/MACE-measure/final_mace_predictions.pkl')
eli5_dlg_quality_df = pd.read_csv('../../data/eli5_ds/annotation-results/MACE-measure/final_mace_rating_predictions.csv')

eli5_annotation_df['exp_type_label'] = eli5_annotation_df['exp_type_label'].fillna('(Y07) Not Explanation')

In [12]:
quality_scores = pd.Series(eli5_dlg_quality_df.rating_label.values, index = eli5_dlg_quality_df.task_id).to_dict()

In [13]:
eli5_annotation_df.exp_act_label.value_counts()

(E03) Provide an explanation                1403
(E04) Ask for an explanation                1178
(E07) Providing Feedback                    1045
(E06) Signaling non-understanding            114
(E05) Signaling understanding                 46
(E08) Providing Assessment                    19
(E10) Introducing Extraneous Information      18
(E02) Testing prior knowledge                 17
(E01) Testing understanding                   17
(E09) Other                                   16
Name: exp_act_label, dtype: int64

In [14]:
eli5_annotation_df = eli5_annotation_df.groupby('task_id').agg({'turn_text': lambda rows: list(rows),
                                                  'topic': lambda rows: list(rows)[0],
                                                  'topic_func_label': lambda rows: list(rows),
                                                  'dlg_act_label': lambda rows: list(rows),
                                                  'exp_type_label' : lambda rows: list(rows),
                                                  'exp_act_label': lambda rows: list(rows)}).reset_index()

eli5_annotation_df['quality'] = eli5_annotation_df.task_id.apply(lambda x: quality_scores[x])

In [ ]:
eli5_annotation_df.head()

In [ ]:
len(eli5_annotation_df)

In [ ]:
eli5_annotation_df['num_turns'] = eli5_annotation_df.turn_text.apply(lambda x: len(x))
eli5_annotation_df['num_tokens'] = eli5_annotation_df.turn_text.apply(lambda turn_txts: len(' '.join([t['text'] for t in turn_txts])))

In [ ]:
eli5_annotation_df.num_tokens.hist()
plt.show()

In [ ]:
len([x for x in eli5_annotation_df.num_tokens if x > 4096]) #number of dialouges longer than 4096

### Experiments:

In [ ]:
# from transformers import LongformerTokenizerFast
# from transformers import LongformerConfig
# from transformers import LongformerTokenizer

# from longformer import LongformerForSequenceClassification
# from transformers import (AutoTokenizer, AutoModelForSequenceClassification, default_data_collator,
#                           PreTrainedModel, BertModel, BertForSequenceClassification,
#                           TrainingArguments, Trainer)

In [ ]:
eli5_annotation_df['turn_text_25'] = eli5_annotation_df.turn_text.apply(lambda turns: turns[:math.ceil(len(turns) * 0.25)])
eli5_annotation_df['turn_text_50'] = eli5_annotation_df.turn_text.apply(lambda turns: turns[:math.ceil(len(turns) * 0.50)])
eli5_annotation_df['turn_text_75'] = eli5_annotation_df.turn_text.apply(lambda turns: turns[:math.ceil(len(turns) * 0.75)])
eli5_annotation_df['turn_text_100']= eli5_annotation_df.turn_text.apply(lambda turns: turns)

In [ ]:
%autoreload

from utils import *

In [ ]:
#RMSE for an avg-baseline
quality_scores = eli5_annotation_df.quality.values
avg_score = np.average(quality_scores)

MSE = np.square(np.subtract(quality_scores,[avg_score] * len(quality_scores))).mean() 
 
RMSE = math.sqrt(MSE)
print("Mean Square Error:", round(MSE, 3))
print("Root Mean Square Error:", round(RMSE, 3))

In [16]:
#tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
#res = preprocess_function(tokenizer, eli5_annotation_df.sample(2), input_clm='turn_text', extra_exp_moves=False, extra_exp_types=False, global_attention=True)

### Train Models on different percentages of the dialogue:

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

#### Longformer

In [22]:
#Evaluate the baseline_pereli5_annotation_dfn different sizes of turns
baseline_rmse_scores = []
for input_clm in ['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    perc = input_clm.split('_')[-1]
    baseline_rmse_scores.append(train_and_evaluate_model('../data/quality_models/longformer-{}'.format(perc), eli5_annotation_df,
                                         'longformer-{}'.format(perc),
                                         input_clm=input_clm,
                                         num_train_epochs=5, lr=2e-5, batch_size=4, n_folds=3, eval_steps=100))

wandb: Currently logged in as: miladalsh-it (milad-it). Use `wandb login --relogin` to force relogin


3    49
1    49
4    49
2    49
5    49
Name: quality, dtype: int64
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Training 108, Valid 28, and Test 68
Training 245, Valid 30, and Test 464


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['longformer.embeddings.exp_type_embeddings.weight', '

Step,Training Loss,Validation Loss,Rmse
100,2.553100,2.492997,1.578923
200,1.709300,2.152284,1.467066
300,0.938400,2.649135,1.627616


***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 464
  Batch size = 4


...

Configuration saved in ../data/quality_models/longformer-100/longformer-2-fold/config.json


{'eval_loss': 2.63857364654541, 'eval_rmse': 1.6243687868118286, 'eval_runtime': 6.6044, 'eval_samples_per_second': 11.053, 'eval_steps_per_second': 2.877, 'epoch': 5.0}


Model weights saved in ../data/quality_models/longformer-100/longformer-2-fold/pytorch_model.bin


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

eval/loss,▆▇█▅▃▂▁▃▃▂▂█
eval/rmse,▆▇█▅▃▃▁▃▃▃▃█
eval/runtime,▅▁▁▁▁▁▁▁▁▁▁█
eval/samples_per_second,▁██████████▄
eval/steps_per_second,▁██████████▄
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/learning_rate,█▇▇▆▅▅▄▃▂▂▁
train/loss,█▇▅▄▃▃▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [25]:
json.dump(baseline_rmse_scores, open('../data/quality_models/baseline_rmse_scores.json', 'w'))

#### LongFormer + exp-moves

In [ ]:
#Evaluate the baseline_pereli5_annotation_dfn different sizes of turns
exp_move_rmse_scores = []
for input_clm in ['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    perc = input_clm.split('_')[-1]
    exp_move_rmse_scores.append(train_and_evaluate_model('../data/quality_models/longformer-exp-moves-{}'.format(perc), eli5_annotation_df,
                                         'longformer-exp-moves-{}'.format(perc),
                                         input_clm=input_clm,
                                        num_train_epochs=5, lr=2e-5, batch_size=4, n_folds=3, extra_exp_moves=True, num_exp_moves=11, eval_steps=100))

loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/859f4633944e1b7e7fa301e72161388cd5903e36385d0ef2917256506bff64c3.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/af6fcabe2bf8cab6f77b20d94ba46a3dbf441ca0549e1f3c852c437b612f5224.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/tokenizer_config.json from cache at None
loading configuration file 

3    49
1    49
4    49
2    49
5    49
Name: quality, dtype: int64
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Training 108, Valid 28, and Test 68
Training 245, Valid 30, and Test 464


loading weights file https://huggingface.co/allenai/longformer-base-4096/resolve/main/pytorch_model.bin from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/a7a586602e625bd012d75abdfcc615f5bb1fe133273845f7381332c634273bd9.dc3a4f03d4ab11f972b126d0e6b67f43e5d9003b3aec54f8e549cc7e2d42398d
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exa

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 464
  Batch size = 4


...

eval/loss,▁
eval/rmse,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


1    188
3    188
2    188
4    188
5    188
Name: quality, dtype: int64
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Training 108, Valid 28, and Test 68
Training 940, Valid 30, and Test 115


loading weights file https://huggingface.co/allenai/longformer-base-4096/resolve/main/pytorch_model.bin from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/a7a586602e625bd012d75abdfcc615f5bb1fe133273845f7381332c634273bd9.dc3a4f03d4ab11f972b126d0e6b67f43e5d9003b3aec54f8e549cc7e2d42398d
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exa

Step,Training Loss,Validation Loss,Rmse
500,1.643700,2.530915,1.590885


***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
Saving model checkpoint to ../data/quality_models/longformer-exp-moves-25/longformer-1-fold/checkpoint-500
Configuration saved in ../data/quality_models/longformer-exp-moves-25/longformer-1-fold/checkpoint-500/config.json
Model weights saved in ../data/quality_models/longformer-exp-moves-25/longformer-1-fold/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../data/quality_models/longformer-exp-moves-25/longformer-1-fold/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../data/quality_models/longformer-exp-moves-25/longformer-1-fold/checkpoint-500/special_tokens_map.json


In [28]:
json.dump(exp_move_rmse_scores, open('../data/quality_models/exp_move_rmse_scores.json', 'w'))

#### LongFormer + exp-types

In [30]:
#Evaluate the approach performance on different sizes of turns
exp_types_rmse_scores = []
for input_clm in ['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    perc = input_clm.split('_')[-1]
    exp_types_rmse_scores.append(train_and_evaluate_model('../data/quality_models/longformer-exp-types-{}'.format(perc), eli5_annotation_df, 
                                       'longformer-exp-types-{}'.format(perc), 
                                        input_clm=input_clm,
                                       num_train_epochs=5, lr=2e-5, batch_size=4, extra_exp_types=True, num_exp_types=8, n_folds=3, eval_steps=100)
    )

loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/859f4633944e1b7e7fa301e72161388cd5903e36385d0ef2917256506bff64c3.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/af6fcabe2bf8cab6f77b20d94ba46a3dbf441ca0549e1f3c852c437b612f5224.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/tokenizer_config.json from cache at None
loading configuration file 

3    49
1    49
4    49
2    49
5    49
Name: quality, dtype: int64
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Training 108, Valid 28, and Test 68
Training 245, Valid 30, and Test 464


loading weights file https://huggingface.co/allenai/longformer-base-4096/resolve/main/pytorch_model.bin from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/a7a586602e625bd012d75abdfcc615f5bb1fe133273845f7381332c634273bd9.dc3a4f03d4ab11f972b126d0e6b67f43e5d9003b3aec54f8e549cc7e2d42398d
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exa

Step,Training Loss,Validation Loss,Rmse
100,2.687900,2.536054,1.592499
200,1.864800,2.224604,1.491511
300,1.097100,2.756962,1.660410


***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 464
  Batch size = 4


...

Configuration saved in ../data/quality_models/longformer-exp-types-100/longformer-2-fold/config.json


{'eval_loss': 2.275981903076172, 'eval_rmse': 1.5086356401443481, 'eval_runtime': 7.6872, 'eval_samples_per_second': 9.496, 'eval_steps_per_second': 2.472, 'epoch': 5.0}


Model weights saved in ../data/quality_models/longformer-exp-types-100/longformer-2-fold/pytorch_model.bin


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

eval/loss,██▃▃▃▅▂▂▁▁▁▆
eval/rmse,██▃▃▃▅▂▂▁▁▁▇
eval/runtime,▁▁▁▁▁▁▁▁▁▁▁█
eval/samples_per_second,███████████▁
eval/steps_per_second,███████████▁
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/learning_rate,█▇▇▆▅▅▄▃▂▂▁
train/loss,█▆▅▄▃▂▂▂▁▁▁
train/total_flos,▁
train/train_loss,▁


In [31]:
json.dump(exp_types_rmse_scores, open('../data/quality_models/exp_types_rmse_scores.json', 'w'))

#### LongFormer + exp-types & Moves

In [32]:
#Evaluate the approach performance on different sizes of turns
exp_types_and_moves_rmse_scores = []
for input_clm in ['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    perc = input_clm.split('_')[-1]
    exp_types_and_moves_rmse_scores.append(train_and_evaluate_model('../data/quality_models/longformer-exp-types-and-moves{}'.format(perc), eli5_annotation_df, 
                                       'longformer-exp-types-and-moves{}'.format(perc), 
                                        input_clm=input_clm,
                                        num_train_epochs=5, lr=2e-5, batch_size=4, extra_exp_types=True, num_exp_types=8, extra_exp_moves=True, num_exp_moves=11, n_folds=3, eval_steps=100)
    )

loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/859f4633944e1b7e7fa301e72161388cd5903e36385d0ef2917256506bff64c3.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/af6fcabe2bf8cab6f77b20d94ba46a3dbf441ca0549e1f3c852c437b612f5224.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/allenai/longformer-base-4096/resolve/main/tokenizer_config.json from cache at None
loading configuration file 

3    49
1    49
4    49
2    49
5    49
Name: quality, dtype: int64
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Using input_clm=turn_text_25
Using global attention
Training 108, Valid 28, and Test 68
Training 245, Valid 30, and Test 464


loading weights file https://huggingface.co/allenai/longformer-base-4096/resolve/main/pytorch_model.bin from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/a7a586602e625bd012d75abdfcc615f5bb1fe133273845f7381332c634273bd9.dc3a4f03d4ab11f972b126d0e6b67f43e5d9003b3aec54f8e549cc7e2d42398d
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exa

Step,Training Loss,Validation Loss,Rmse
100,2.609300,1.747996,1.322118
200,1.332300,2.564864,1.601519
300,0.601400,2.845880,1.686974


***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 464
  Batch size = 4


...

***** Running Evaluation *****
  Num examples = 37
  Batch size = 4
***** Running Evaluation *****
  Num examples = 37
  Batch size = 4
***** Running Evaluation *****
  Num examples = 37
  Batch size = 4
***** Running Evaluation *****
  Num examples = 37
  Batch size = 4
***** Running Evaluation *****
  Num examples = 37
  Batch size = 4
Saving model checkpoint to ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/checkpoint-500
Configuration saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/checkpoint-500/config.json
Model weights saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/checkpoint-500/special_tokens_map.json
*

{'eval_loss': 1.751179575920105, 'eval_rmse': 1.3233213424682617, 'eval_runtime': 6.3549, 'eval_samples_per_second': 11.487, 'eval_steps_per_second': 2.99, 'epoch': 5.0}


Configuration saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/config.json
Model weights saved in ../data/quality_models/longformer-exp-types-and-moves100/longformer-2-fold/pytorch_model.bin


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

eval/loss,█▅▄▃▂▂▂▃▁▁▁▅
eval/rmse,█▅▄▃▂▂▂▃▁▁▁▅
eval/runtime,▁▁▁▁▁▁▁▁▁▁▁█
eval/samples_per_second,███▇███████▁
eval/steps_per_second,███████████▁
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
train/learning_rate,█▇▇▆▅▅▄▃▂▂▁
train/loss,█▅▄▄▃▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [34]:
json.dump(exp_types_and_moves_rmse_scores, open('../data/quality_models/exp_types_and_moves_rmse_scores.json', 'w'))

#### HAT model:

In [41]:
model_name_or_path = 'kiddothe2b/hierarchical-transformer-base-4096'

In [42]:
#Evaluate the approach performance on different sizes of turns
hat_rmse_scores = []
for input_clm in ['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    eli5_annotation_df['input_texts'] = eli5_annotation_df[input_clm].apply(lambda row: [x['text'] for x in row])
    eli5_annotation_df['labels'] = eli5_annotation_df.quality
    perc = input_clm.split('_')[-1]
    hat_rmse_scores = train_and_evaluate_hat_model(model_name_or_path, '../data/quality_models/hat-model-{}'.format(perc), 
                                                         eli5_annotation_df, 'hat-model-{}'.format(perc), num_train_epochs=5, 
                                                         lr=5e-5, batch_size=8,  n_folds=5, eval_steps=50)

4    77
2    77
1    77
5    77
3    77
Name: quality, dtype: int64


  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

The following columns in the evaluation set  don't have a corresponding argument in `MyHATForSequenceClassification.forward` and have been ignored: input_texts, __index_level_0__. If input_texts, __index_level_0__ are not expected by `MyHATForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 35
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `MyHATForSequenceClassification.forward` and have been ignored: input_texts, __index_level_0__. If input_texts, __index_level_0__ are not expected by `MyHATForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 35
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `MyHATForSequenceClassification.forward` and have been ignored: input_texts, __index_level_0__. If input_texts, __index_level_0__ are not expected by `MyH

{'eval_loss': 2.4996938705444336, 'eval_rmse': 1.5810418128967285, 'eval_runtime': 0.3053, 'eval_samples_per_second': 144.128, 'eval_steps_per_second': 19.654, 'epoch': 5.0}


Configuration saved in ../data/quality_models/hat-model-100/hat-former-4-fold/config.json
Model weights saved in ../data/quality_models/hat-model-100/hat-former-4-fold/pytorch_model.bin


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

eval/loss,▂█▄▂▃▄▁▃▃▂▄▃▃
eval/rmse,▃█▄▂▃▄▁▃▃▂▄▃▃
eval/runtime,▁▂▁▂▁▂▁▂▁▁▁▂█
eval/samples_per_second,▇▇█▅▇▅▇▆█▇▇▆▁
eval/steps_per_second,█▇█▅▇▆▇▆█▇▇▆▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇████
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁
train/loss,█▆▆▃▃▃▂▂▁▂▁▁
train/total_flos,▁
train/train_loss,▁


In [19]:
json.dump(hat_rmse_scores, open('../data/quality_models/hat_rmse_scores.json', 'w'))

#### HAT model + exp-moves:

In [48]:
extra_feats_clm = 'exp_act_label'
eli5_annotation_df[extra_feats_clm] = eli5_annotation_df[extra_feats_clm].apply(lambda row: [int(x[2:4]) for x in row])

In [65]:
#Evaluate the approach performance on different sizes of turns
hat_rmse_scores = []
for input_clm in [ 'turn_text_100']: #['turn_text_25', 'turn_text_50', 'turn_text_75', 'turn_text_100']:
    eli5_annotation_df['input_texts'] = eli5_annotation_df[input_clm].apply(lambda row: [x['text'] for x in row])
    eli5_annotation_df['labels'] = eli5_annotation_df.quality
    perc = input_clm.split('_')[-1]
    hat_rmse_scores = train_and_evaluate_hat_model(model_name_or_path, '../data/quality_models/hat-model-exp-moves-test-3-{}'.format(perc), 
                                                         eli5_annotation_df, 'hat-model-exp-moves-test-3-{}'.format(perc), num_train_epochs=5, 
                                                         lr=5e-5, batch_size=8,  n_folds=5, eval_steps=50, extra_encoder=True, extra_feats_clm=extra_feats_clm,
                                                  flow_model_hidden_size=256, nhead=6, nlayers=6)

loading configuration file https://huggingface.co/kiddothe2b/hierarchical-transformer-base-4096/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current/sile2804/.cache/huggingface/transformers/67fff7e4e0a33106e11c355770df4420b2385850d788d1b7b549eb8d64b4f3d1.ef5baf2314e8ee19e0e25bc7741c47229b1ac5c08ae795cbc2d27f5f2c19535c
Model config HATConfig {
  "_name_or_path": "kiddothe2b/hierarchical-transformer-base-4096",
  "architectures": [
    "HATForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "auto_map": {
    "AutoConfig": "configuration_hat.HATConfig",
    "AutoModel": "modelling_hat.HATModel",
    "AutoModelForMaskedLM": "modelling_hat.HATForMaskedLM",
    "AutoModelForMultipleChoice": "modelling_hat.HATForMultipleChoice",
    "AutoModelForQuestionAnswering": "modelling_hat.HATForQuestionAnswering",
    "AutoModelForSequenceClassification": "modelling_hat.HATForSequenceClassification",
    "AutoModelForTokenClassification": "modelling_hat.HATForTokenClassific

4    77
2    77
1    77
5    77
3    77
Name: quality, dtype: int64


  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1 [00:00<?, ?ba/s]

...

Configuration saved in ../data/quality_models/hat-model-exp-moves-test-3-100/hat-former-4-fold/config.json


{'eval_loss': 2.265375852584839, 'eval_rmse': 1.5051167011260986, 'eval_runtime': 0.2756, 'eval_samples_per_second': 159.631, 'eval_steps_per_second': 21.768, 'epoch': 5.0}


Model weights saved in ../data/quality_models/hat-model-exp-moves-test-3-100/hat-former-4-fold/pytorch_model.bin


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

eval/loss,█▆▄▁▁▃▂▃▂▂▁▂▂
eval/rmse,█▆▄▁▁▃▂▄▂▂▁▂▃
eval/runtime,▂▆▃▄▂▄▁▄▂▄▁█▇
eval/samples_per_second,▆▃▆▅▇▄█▄▇▄▇▁▇
eval/steps_per_second,▆▃▆▅▇▄█▄▇▄▇▁▆
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇████
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁
train/loss,█▇▆▄▄▃▂▂▂▂▁▁
train/total_flos,▁
train/train_loss,▁


### Printing Results:

In [66]:
print(tabulate([
        ['Longformer']  + [load_results('../data/quality_models/longformer-25/')[1], load_results('../data/quality_models/longformer-50/')[1], load_results('../data/quality_models/longformer-75/')[1], load_results('../data/quality_models/longformer-100/')[1]],
        ['Exp Moves'] + [load_results('../data/quality_models/longformer-exp-moves-25/')[1], load_results('../data/quality_models/longformer-exp-moves-50/')[1], load_results('../data/quality_models/longformer-exp-moves-75/')[1], load_results('../data/quality_models/longformer-exp-moves-100/')[1]],
        ['Exp Types'] + [load_results('../data/quality_models/longformer-exp-types-25/')[1], load_results('../data/quality_models/longformer-exp-types-50/')[1], load_results('../data/quality_models/longformer-exp-types-75/')[1], load_results('../data/quality_models/longformer-exp-types-100/')[1]],
        ['Exp Moves and Types'] + [load_results('../data/quality_models/longformer-exp-types-and-moves25/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves50/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves75/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves100/')[1]],
        ['HAT'] + [load_results('../data/quality_models/hat-model-25/')[1], load_results('../data/quality_models/hat-model-50/')[1], load_results('../data/quality_models/hat-model-75')[1], load_results('../data/quality_models/hat-model-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-25/')[1], load_results('../data/quality_models/hat-model-exp-moves-50/')[1], load_results('../data/quality_models/hat-model-exp-moves-75')[1], load_results('../data/quality_models/hat-model-exp-moves-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-1-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-2-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-3-100/')[1]],
], headers=['Approach', '@25%', '@50%', '@75%', '@100%']))

Approach                @25%     @50%     @75%    @100%
-------------------  -------  -------  -------  -------
Longformer           1.67567  1.58526  1.57192  1.5375
Exp Moves            1.66363  1.57336  1.45779  1.43321
Exp Types            1.65409  1.58038  1.542    1.44747
Exp Moves and Types  1.6653   1.58473  1.58456  1.41372
HAT                  1.75281  1.59851  1.53645  1.43273
HAT + Exp-moves      1.72779  1.692    1.52813  1.5525
HAT + Exp-moves      1.56315
HAT + Exp-moves      1.47224
HAT + Exp-moves      1.65892
HAT + Exp-moves      1.65892


In [62]:
print(tabulate([
        ['Longformer']  + [load_results('../data/quality_models/longformer-25/')[1], load_results('../data/quality_models/longformer-50/')[1], load_results('../data/quality_models/longformer-75/')[1], load_results('../data/quality_models/longformer-100/')[1]],
        ['Exp Moves'] + [load_results('../data/quality_models/longformer-exp-moves-25/')[1], load_results('../data/quality_models/longformer-exp-moves-50/')[1], load_results('../data/quality_models/longformer-exp-moves-75/')[1], load_results('../data/quality_models/longformer-exp-moves-100/')[1]],
        ['Exp Types'] + [load_results('../data/quality_models/longformer-exp-types-25/')[1], load_results('../data/quality_models/longformer-exp-types-50/')[1], load_results('../data/quality_models/longformer-exp-types-75/')[1], load_results('../data/quality_models/longformer-exp-types-100/')[1]],
        ['Exp Moves and Types'] + [load_results('../data/quality_models/longformer-exp-types-and-moves25/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves50/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves75/')[1], load_results('../data/quality_models/longformer-exp-types-and-moves100/')[1]],
        ['HAT'] + [load_results('../data/quality_models/hat-model-25/')[1], load_results('../data/quality_models/hat-model-50/')[1], load_results('../data/quality_models/hat-model-75')[1], load_results('../data/quality_models/hat-model-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-25/')[1], load_results('../data/quality_models/hat-model-exp-moves-50/')[1], load_results('../data/quality_models/hat-model-exp-moves-75')[1], load_results('../data/quality_models/hat-model-exp-moves-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-100/')[1]],
        ['HAT + Exp-moves'] + [load_results('../data/quality_models/hat-model-exp-moves-test-1-100/')[1]],
], headers=['Approach', '@25%', '@50%', '@75%', '@100%']))

Approach                @25%     @50%     @75%    @100%
-------------------  -------  -------  -------  -------
Longformer           1.67567  1.58526  1.57192  1.5375
Exp Moves            1.66363  1.57336  1.45779  1.43321
Exp Types            1.65409  1.58038  1.542    1.44747
Exp Moves and Types  1.6653   1.58473  1.58456  1.41372
HAT                  1.75281  1.59851  1.53645  1.43273
HAT + Exp-moves      1.72779  1.692    1.52813  1.5525
HAT + Exp-moves      1.56315
HAT + Exp-moves      1.47224
